# Template del Prompt para el Trainig


In [ ]:
TEXT2SQL_TRAINING_PROMPT_TEMPLATE = """\
### Instrucciones:
{system_message}

### Pregunta:
{input}

### Contexto:
{context}

### Respuesta:
{response}
"""


In [ ]:
SYSTEM_MESSAGE = (
    "Eres un modelo especializado en convertir lenguaje natural a SQL. "
    "Dada una pregunta en lenguaje natural y un contexto opcional de base de datos (esquema de tablas, tipos de datos, etc.), "
    "genera una consulta SQL válida y eficiente. "
    "Asegúrate de usar las columnas y tablas correctas, y evita consultas innecesariamente complejas."
    "Responde solo con la consulta SQL y no incluyas explicaciones adicionales."
)

In [ ]:
def create_sql_prompt(sample):
  full_prompt = TEXT2SQL_TRAINING_PROMPT_TEMPLATE.format(
      system_message = SYSTEM_MESSAGE,
      input = sample["question"],
      context = sample["context"],
      response = sample["answer"]
  )

  return {"text" : full_prompt}

# Carga de Datos

In [ ]:
!pip install datasets trl

In [ ]:
from datasets import  DatasetDict

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df=DatasetDict.load_from_disk('/content/drive/MyDrive/IA/datasets/sql_dataset')

In [ ]:
df['train'][0]

{'answer': "SELECT AVG(tiempoRespuesta_min) FROM rpt_actual_casos WHERE Estado = 'En Proceso'",
 'question': "¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'?",
 'context': 'CREATE TABLE rpt_actual_casos (\n            IdCaso INT NOT NULL,\n            idTicket BIGINT NULL,\n            idCliente BIGINT NULL,\n            canal VARCHAR(45) NULL,\n            estadoTicket INT NULL,\n            Estado VARCHAR(50) NULL,\n            frecepcion TIMESTAMP NULL,\n            fgestion TIMESTAMP NULL,\n            fechaEjecutivo TIMESTAMP NULL,\n            fechaTicket TIMESTAMP NULL,\n            fechaCierreRequerimiento TIMESTAMP NULL,\n            tiempoRespuesta_min INT NULL,\n            nombre_area VARCHAR(200) NULL,\n            nombre_categoria VARCHAR(500) NULL,\n            descripcionProceso VARCHAR(100) NULL,\n            descripcionSubProceso VARCHAR(100) NULL,\n            mensajeTicket TEXT NULL,\n            scriptFase NVARCHAR(100)

In [ ]:
create_sql_prompt(df["train"][0])

{'text': "### Instrucciones:\nEres un modelo especializado en convertir lenguaje natural a SQL. Dada una pregunta en lenguaje natural y un contexto opcional de base de datos (esquema de tablas, tipos de datos, etc.), genera una consulta SQL válida y eficiente. Asegúrate de usar las columnas y tablas correctas, y evita consultas innecesariamente complejas.Responde solo con la consulta SQL y no incluyas explicaciones adicionales.\n\n### Pregunta:\n¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'?\n\n### Contexto:\nCREATE TABLE rpt_actual_casos (\n            IdCaso INT NOT NULL,\n            idTicket BIGINT NULL,\n            idCliente BIGINT NULL,\n            canal VARCHAR(45) NULL,\n            estadoTicket INT NULL,\n            Estado VARCHAR(50) NULL,\n            frecepcion TIMESTAMP NULL,\n            fgestion TIMESTAMP NULL,\n            fechaEjecutivo TIMESTAMP NULL,\n            fechaTicket TIMESTAMP NULL,\n            fechaCierreReq

In [ ]:
sql_dataset = df.map(create_sql_prompt)

In [ ]:
sql_dataset['train'][0]

{'answer': "SELECT AVG(tiempoRespuesta_min) FROM rpt_actual_casos WHERE Estado = 'En Proceso'",
 'question': "¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'?",
 'context': 'CREATE TABLE rpt_actual_casos (\n            IdCaso INT NOT NULL,\n            idTicket BIGINT NULL,\n            idCliente BIGINT NULL,\n            canal VARCHAR(45) NULL,\n            estadoTicket INT NULL,\n            Estado VARCHAR(50) NULL,\n            frecepcion TIMESTAMP NULL,\n            fgestion TIMESTAMP NULL,\n            fechaEjecutivo TIMESTAMP NULL,\n            fechaTicket TIMESTAMP NULL,\n            fechaCierreRequerimiento TIMESTAMP NULL,\n            tiempoRespuesta_min INT NULL,\n            nombre_area VARCHAR(200) NULL,\n            nombre_categoria VARCHAR(500) NULL,\n            descripcionProceso VARCHAR(100) NULL,\n            descripcionSubProceso VARCHAR(100) NULL,\n            mensajeTicket TEXT NULL,\n            scriptFase NVARCHAR(100)

#### Cargar el modelo a afinar y preprocesarlo

Usaremos mbart-large-50, que es multilingüe y T5 realizaremos fine-tuning en tareas de conversión de texto a SQL

In [ ]:
from transformers import AutoTokenizer,MBartForConditionalGeneration
model_name = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

MBartScaledWordEmbedding(250054, 1024, padding_idx=1)

In [ ]:
from transformers import pipeline, set_seed, GenerationConfig

class Generador:
    def __init__(self, model, tokenizer, seed=42):
        self.TEXT2SQL_INFERENCE_PROMPT_TEMPLATE = """\
            ### Instrucciones:
            {system_message}

            ### Pregunta:
            {input}

            ### Contexto:
            {context}

            ### Respuesta:
        """
        self.SYSTEM_MESSAGE = (
            "Eres un modelo especializado en convertir lenguaje natural a SQL. "
            "Dada una pregunta en lenguaje natural y un contexto opcional de base de datos (esquema de tablas, tipos de datos, etc.), "
            "genera una consulta SQL válida y eficiente. "
            "Asegúrate de usar las columnas y tablas correctas, y evita consultas innecesariamente complejas."
            "Responde solo con la consulta SQL y no incluyas explicaciones adicionales."
        )
        set_seed(seed)
        self.model = model
        self.tokenizer = tokenizer
        self.generator = pipeline('text2text-generation', model=self.model, tokenizer=self.tokenizer)

    def get_generation_config(self, max_tokens=512, top_k=512, temperature=1e-4):
        return GenerationConfig(
            max_new_tokens=max_tokens,
            do_sample=True,
            top_k=top_k,
            temperature=temperature,
            eos_token_id=self.model.config.eos_token_id,
        )

    def generate_sample(self, sample):
        prompt_package = self.create_sql_prompt_and_response(sample)
        config = self.get_generation_config()

        # Generar la respuesta del modelo
        generation = self.generator(prompt_package["full_prompt"], generation_config=config)
        generated_text = generation[0]["generated_text"]

        # Intentar limpiar la salida
        if generated_text.startswith(prompt_package["full_prompt"]):
            generated_text = generated_text[len(prompt_package["full_prompt"]):].strip()

        return {
            "question": sample["question"],
            "ground_truth": prompt_package["ground_truth"],
            "model_response": generated_text
        }

    def create_sql_prompt_and_response(self, sample):
        full_prompt = self.TEXT2SQL_INFERENCE_PROMPT_TEMPLATE.format(
            system_message=self.SYSTEM_MESSAGE,
            input=sample["question"],
            context=sample["context"]
        )

        return {
            "full_prompt": full_prompt,
            "ground_truth": sample["answer"]
        }


In [ ]:
import json

generador = Generador(model, tokenizer)
resultado = generador.generate_sample(sql_dataset['test'][10])
print(json.dumps(resultado, indent=4, ensure_ascii=False))

Device set to use cuda:0


{
    "question": "¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'?",
    "ground_truth": "SELECT AVG(tiempoRespuesta_min) FROM rpt_actual_casos WHERE Estado = 'En Proceso'",
    "model_response": "### Instrucciones: Eres un modelo especializado en convertir lenguaje natural a SQL. Dada una pregunta en lenguaje natural y un contexto opcional de base de datos (esquema de tablas, tipos de datos, etc.), genera una consulta SQL válida y eficiente. Asegúrate de usar las columnas y tablas correctas, y evita consultas innecesariamente complejas.Responde solo con la consulta SQL y no incluyas explicaciones adicionales. ### Pregunta: ¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'? ### Contexto: CREATE TABLE rpt_actual_casos ( IdCaso INT NOT NULL, idTicket BIGINT NULL, idCliente BIGINT NULL, canal VARCHAR(45) NULL, estadoTicket INT NULL, Estado VARCHAR(50) NULL, frecepcion TIMESTAMP NULL, fgestion TIMESTAM

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer
training_args = TrainingArguments(
 per_device_train_batch_size=4,
 gradient_accumulation_steps=4,
 gradient_checkpointing=True,
 max_grad_norm= 0.3,
 ##num_train_epochs=5,
 max_steps=300,
 learning_rate=5e-5,
 save_total_limit=3,
 logging_steps=10,
 weight_decay=0.1,
 output_dir="/content/drive/MyDrive/IA/outputs/model/t5_text2sql_model_v1",
 optim="adamw_torch",
 lr_scheduler_type="cosine",
 eval_strategy="steps",
 eval_steps=100,
 warmup_ratio=0.05,
 fp16=False,
 report_to="none"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=sql_dataset["train"],
    eval_dataset=sql_dataset["validation"],
    processing_class=tokenizer,
    args=training_args,

)




In [ ]:
!pip install bitsandbytes

In [ ]:
model.config.use_cache = False

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
trainer.optimizer = optimizer

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/IA/outputs/model/t5_text2sql_model_v1")
tokenizer.save_pretrained("/content/drive/MyDrive/IA/outputs/model/t5_text2sql_tokenizer")

In [19]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Directorios de modelo y tokenizer
model_dir = "./outputs/model/t5_v1/checkpoint-600"
tokenizer_dir = "./outputs/model/t5_v1/checkpoint-600"

# Cargar el modelo y el tokenizer
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)

# Mover el modelo a GPU si está disponible
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [24]:
import json

generador = Generador(model, tokenizer)
resultado = generador.generate_sample(sql_dataset['test'][10])
print(json.dumps(resultado, indent=4, ensure_ascii=False))

Device set to use cuda:0


{
    "question": "¿Cuál es el promedio de tiempo de respuesta en minutos para los casos en estado 'En Proceso'?",
    "ground_truth": "SELECT AVG(tiempoRespuesta_min) FROM rpt_actual_casos WHERE Estado = 'En Proceso'",
    "model_response": "Eres un modelo especializado en convertir lenguaje natural a SQL. Dado una pregunta y un contexto opcional de base de datos"
}
